# Práctica 1 - Regresión

Vamos a trabajar con un conjunto de datos que representa una serie de billetes de renfe. Entre sus parámetros se encuentran:




*   **id**: identificador del billete
*   **company**: empresa
*   **origin**: lugar de salida del tren
*   **destination**: lugar de llegada del tren
*   **departure**: hora de salida del tren
*   **arrival**: hora de llegada del tren
*   **duration**: duración del trayecto
*   **vehicle_type**: tipo de tren
*   **vehicle_class**: clase del pasajero
*   **price**: precio (variable objetivo, a predecir)





In [1]:
test_url = "https://gist.githubusercontent.com/w-dan/a3c63b2aed66a5edd1288c1d3006fc79/raw/9dcc831af941428edad3ef285729e4ef5a63a9fb/renfe-test.csv"
train_url = "https://gist.githubusercontent.com/w-dan/a3c63b2aed66a5edd1288c1d3006fc79/raw/9dcc831af941428edad3ef285729e4ef5a63a9fb/renfe-train.csv"

# Preprocesamiento

Paso 1: cargamos los datos a dataframes, comprobamos su estado

In [39]:
import pandas as pd
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

In [11]:
test_df

,id,company,origin,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,seats,meta,insert_date
0,3002,renfe,MADRID,BARCELONA,2019-04-28 08:30:00,2019-04-28 11:15:00,2.75,AVE,Turista,75.40,Promo,NaN,{},2019-04-11 22:00:09
1,3003,renfe,MADRID,BARCELONA,2019-04-28 20:00:00,2019-04-28 22:30:00,2.50,AVE,Preferente,115.65,Promo,NaN,{},2019-04-11 22:00:09
2,3004,renfe,MADRID,BARCELONA,2019-04-28 09:30:00,2019-04-28 12:34:00,3.07,AVE,Turista Plus,90.50,Promo,NaN,{},2019-04-11 22:00:09
3,3005,renfe,MADRID,VALENCIA,2019-06-03 12:21:00,2019-06-03 19:04:00,6.72,REGIONAL,Turista,28.35,Adulto ida,NaN,{},2019-04-11 22:00:09
4,3006,renfe,MADRID,BARCELONA,2019-04-28 10:30:00,2019-04-28 13:15:00,2.75,AVE,Turista,85.10,Promo,NaN,{},2019-04-11 22:00:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,4996,renfe,MADRID,BARCELONA,2019-04-12 10:30:00,2019-04-12 13:15:00,2.75,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1995,4997,renfe,MADRID,BARCELONA,2019-04-12 16:30:00,2019-04-12 19:15:00,2.75,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1996,4998,renfe,MADRID,BARCELONA,2019-04-12 11:30:00,2019-04-12 14:40:00,3.17,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1997,4999,renfe,MADRID,BARCELONA,2019-04-12 17:30:00,2019-04-12 20:40:00,3.17,AVE,Turista,NaN,Flexible,NaN,{},2019-04-11 22:07:27


Ahora que tenemos en mente la forma de estos datos, podemos empezar a pensar. Queremos predecir el precio de un billete de tren en base a información relativa al viaje (como puede ser la duración, la clase o el tipo de tren). ¿Qué columnas nos sobran a primera vista?
* **id**, no aporta nada un identificador para estudiar un precio
* **company**, por tener un único valor
* **origin**, también tiene un único valor
* **seats** y **meta**, porque están completamente rotas (perdón)
* **insert_date**, es una fecha de inserción a la base de datos, queremos saber el precio en base a datos de los trenes, así que nos sobra.

In [40]:
# ignoremos esto de momento, es de cara a subir un resultado a kaggle
submission = test_df[['id']].copy()

train_df = train_df.drop(['id', 'company', 'seats', 'meta', 'origin', 'insert_date'], axis=1)
test_df = test_df.drop(['id', 'company', 'seats', 'meta', 'origin', 'insert_date'], axis=1)

train_df

,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare
0,BARCELONA,2019-04-18 05:50:00,2019-04-18 08:55:00,3.08,AVE,Preferente,68.95,Promo
1,BARCELONA,2019-04-18 13:25:00,2019-04-18 16:24:00,2.98,AVE-TGV,Turista,107.70,Flexible
2,BARCELONA,2019-04-18 06:30:00,2019-04-18 09:20:00,2.83,AVE,Turista,75.40,Promo
3,BARCELONA,2019-04-18 15:30:00,2019-04-18 18:40:00,3.17,AVE,Preferente,NaN,Promo
4,BARCELONA,2019-04-18 07:00:00,2019-04-18 09:30:00,2.50,AVE,Turista Plus,106.75,Promo
...,...,...,...,...,...,...,...,...
2995,VALENCIA,2019-06-03 16:05:00,2019-06-03 22:47:00,6.70,REGIONAL,Turista,28.35,Adulto ida
2996,BARCELONA,2019-04-28 14:30:00,2019-04-28 17:21:00,2.85,AVE,Turista,107.70,Flexible
2997,BARCELONA,2019-04-28 06:20:00,2019-04-28 09:29:00,3.15,AVE,Turista,49.55,Promo
2998,BARCELONA,2019-04-28 19:30:00,2019-04-28 22:40:00,3.17,AVE,Turista,85.10,Promo


Bien, ahora que tenemos más limpios los datos, llega la hora de hacernos otra pregunta: *¿cómo podría sacar el máximo provecho de los datos que tengo aquí?*
Tenemos datos compuestos como la fecha (tiene mes, día y año, habrá que separar), además de datos categóricos de tipo string. Recordemos que las regresiones, generalmente, necesitan datos continuos y numéricos. Esto tenemos que tratarlo.<br><br>
Un buen primer paso es empezar con las transformaciones más sencillas, podemos simplemente transformar a números las columnas **vehicle_type**, **vehicle_class**, **fare**, y **destination**, ya que no toman muchos valores distintos... Espera, ¿podemos? NUNCA debemos olvidarnos de comprobar la cantidad de valores distintos antes de llevar a cabo estas transformaciones:

In [17]:
print("La columna vehicle_type tiene: ", len(train_df.vehicle_type.unique()), " valores distintos")
print("La columna vehicle_class tiene: ", len(train_df.vehicle_class.unique()), " valores distintos")
print("La columna fare tiene: ", len(train_df.fare.unique()), " valores distintos")
print("La columna destination tiene: ", len(train_df.destination.unique()), " valores distintos")

La columna vehicle_type tiene:  10  valores distintos
La columna vehicle_class tiene:  5  valores distintos
La columna fare tiene:  5  valores distintos
La columna destination tiene:  3  valores distintos


Bien, tenemos una cantidad baja y manejable de valores distintos, conque podemos transformarlos a números sin mayor problema. Confirmamos la naturaleza discreta o categórica de estas columnas por la baja cantidad de valores distintos.

Pero todavía no podemos quedarnos tranquilos... Estos datos traen algún valor "nan". ¿Cómo podemos deshacernos de este inconveniente?

In [29]:
print("vechicle_type: ", train_df.vehicle_type.unique(), " valores nulos: ", train_df.vehicle_type.isnull().sum())
print("vechicle_class: ", train_df.vehicle_class.unique(), " valores nulos: ", train_df.vehicle_class.isnull().sum())
print("fare: ", train_df.fare.unique(), " valores nulos: ", train_df.fare.isnull().sum())
print("destination: ", train_df.destination.unique(), " valores nulos: ", train_df.destination.isnull().sum())

vechicle_type:  [2. 5. 8. 1. 6. 0. 7. 9. 4. 3.]  valores nulos:  0
vechicle_class:  ['Preferente' 'Turista' 'Turista Plus' nan 'Turista con enlace']  valores nulos:  4
fare:  ['Promo' 'Flexible' nan 'Adulto ida' 'Promo +']  valores nulos:  4
destination:  ['BARCELONA' 'SEVILLA' 'VALENCIA']  valores nulos:  0


Tanto **vehicle_class** como **fare** contienen valores nulos... (Vaya mierda de datos os pido perdón pero así aprendemos supongo)
Bueno, hay que rellenar, hagamos un apaño algo cutre...

In [41]:
train_df['vehicle_class'] = train_df['vehicle_class'].fillna("Ninguno")
train_df['fare'] = train_df['fare'].fillna("Ninguno")

print("vechicle_class: ", train_df.vehicle_class.unique(), " valores nulos: ", train_df.vehicle_class.isnull().sum())
print("fare: ", train_df.fare.unique(), " valores nulos: ", train_df.fare.isnull().sum())

vechicle_class:  ['Preferente' 'Turista' 'Turista Plus' 'Ninguno' 'Turista con enlace']  valores nulos:  0
fare:  ['Promo' 'Flexible' 'Ninguno' 'Adulto ida' 'Promo +']  valores nulos:  0


¡Arreglado! Perdón de nuevo...
Sigamos, recordemos que íbamos a ir convirtiendo a números nuestras variables categóricas.

In [42]:
from sklearn.preprocessing import OrdinalEncoder

# transformamos (de una en una, sé que es un poco guarrada, pero por legibilidad, para que nadie se pierda)
train_df['vehicle_type'] = OrdinalEncoder().fit_transform(train_df[['vehicle_type']])
train_df['vehicle_class'] = OrdinalEncoder().fit_transform(train_df[['vehicle_class']])
train_df['fare'] = OrdinalEncoder().fit_transform(train_df[['fare']])
train_df['destination'] = OrdinalEncoder().fit_transform(train_df[['destination']])


# repetimos para el conjunto de test
test_df['vehicle_type'] = OrdinalEncoder().fit_transform(test_df[['vehicle_type']])
test_df['vehicle_class'] = OrdinalEncoder().fit_transform(test_df[['vehicle_class']])
test_df['fare'] = OrdinalEncoder().fit_transform(test_df[['fare']])
test_df['destination'] = OrdinalEncoder().fit_transform(test_df[['destination']])

test_df

,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare
0,0.0,2019-04-28 08:30:00,2019-04-28 11:15:00,2.75,2.0,1.0,75.40,2.0
1,0.0,2019-04-28 20:00:00,2019-04-28 22:30:00,2.50,2.0,0.0,115.65,2.0
2,0.0,2019-04-28 09:30:00,2019-04-28 12:34:00,3.07,2.0,2.0,90.50,2.0
3,2.0,2019-06-03 12:21:00,2019-06-03 19:04:00,6.72,9.0,1.0,28.35,0.0
4,0.0,2019-04-28 10:30:00,2019-04-28 13:15:00,2.75,2.0,1.0,85.10,2.0
...,...,...,...,...,...,...,...,...
1994,0.0,2019-04-12 10:30:00,2019-04-12 13:15:00,2.75,2.0,1.0,107.70,1.0
1995,0.0,2019-04-12 16:30:00,2019-04-12 19:15:00,2.75,2.0,1.0,107.70,1.0
1996,0.0,2019-04-12 11:30:00,2019-04-12 14:40:00,3.17,2.0,1.0,107.70,1.0
1997,0.0,2019-04-12 17:30:00,2019-04-12 20:40:00,3.17,2.0,1.0,NaN,1.0


Todo números, ahora podríamos pasarle estas columnas (salvo **departure** y **arrival**, que son fechas) a un regresor.
Aún así, vamos a intentar un último paso para este preprocesamiento, vamos a intentar aislar el mes, el día y la hora (sólo la hora, sin minutos ni segundos, etc.).

In [69]:
def obten_dia(fecha):
  dia = fecha.split(" ")[0]
  return dia

def obten_hora(fecha):
  hora = fecha.split(" ")[1]
  return hora

In [50]:
print(type(train_df['departure'].iloc[100]))

<class 'str'>


In [74]:
train_df.drop(['departure_day'], axis=1)
train_df.drop(['departure_time'], axis=1)

,departure_day,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,departure_date
0,2019-04-18,0.0,2019-04-18 05:50:00,2019-04-18 08:55:00,3.08,2.0,1.0,68.95,3.0,2
1,2019-04-18,0.0,2019-04-18 13:25:00,2019-04-18 16:24:00,2.98,5.0,2.0,107.70,1.0,2
2,2019-04-18,0.0,2019-04-18 06:30:00,2019-04-18 09:20:00,2.83,2.0,2.0,75.40,3.0,2
3,2019-04-18,0.0,2019-04-18 15:30:00,2019-04-18 18:40:00,3.17,2.0,1.0,NaN,3.0,2
4,2019-04-18,0.0,2019-04-18 07:00:00,2019-04-18 09:30:00,2.50,2.0,3.0,106.75,3.0,2
...,...,...,...,...,...,...,...,...,...,...
2995,2019-06-03,2.0,2019-06-03 16:05:00,2019-06-03 22:47:00,6.70,9.0,2.0,28.35,0.0,2
2996,2019-04-28,0.0,2019-04-28 14:30:00,2019-04-28 17:21:00,2.85,2.0,2.0,107.70,1.0,2
2997,2019-04-28,0.0,2019-04-28 06:20:00,2019-04-28 09:29:00,3.15,2.0,2.0,49.55,3.0,2
2998,2019-04-28,0.0,2019-04-28 19:30:00,2019-04-28 22:40:00,3.17,2.0,2.0,85.10,3.0,2


In [75]:
train_df.insert(loc=0, column='departure_day', value="")
train_df.insert(loc=1, column='departure_time', value="")

ValueError: ignored

In [76]:
for i in range(0, len(train_df['departure'])):
  train_df['departure_time'].iloc[i] = obten_hora(train_df['departure'].iloc[i])


for i in range(0, len(train_df['departure'])):
  train_df['departure_day'].iloc[i] = obten_dia(train_df['departure'].iloc[i])

train_df

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,departure_day,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,departure_date,departure_time
0,2019-04-18,0.0,2019-04-18 05:50:00,2019-04-18 08:55:00,3.08,2.0,1.0,68.95,3.0,2,05:50:00
1,2019-04-18,0.0,2019-04-18 13:25:00,2019-04-18 16:24:00,2.98,5.0,2.0,107.70,1.0,2,13:25:00
2,2019-04-18,0.0,2019-04-18 06:30:00,2019-04-18 09:20:00,2.83,2.0,2.0,75.40,3.0,2,06:30:00
3,2019-04-18,0.0,2019-04-18 15:30:00,2019-04-18 18:40:00,3.17,2.0,1.0,NaN,3.0,2,15:30:00
4,2019-04-18,0.0,2019-04-18 07:00:00,2019-04-18 09:30:00,2.50,2.0,3.0,106.75,3.0,2,07:00:00
...,...,...,...,...,...,...,...,...,...,...,...
2995,2019-06-03,2.0,2019-06-03 16:05:00,2019-06-03 22:47:00,6.70,9.0,2.0,28.35,0.0,2,16:05:00
2996,2019-04-28,0.0,2019-04-28 14:30:00,2019-04-28 17:21:00,2.85,2.0,2.0,107.70,1.0,2,14:30:00
2997,2019-04-28,0.0,2019-04-28 06:20:00,2019-04-28 09:29:00,3.15,2.0,2.0,49.55,3.0,2,06:20:00
2998,2019-04-28,0.0,2019-04-28 19:30:00,2019-04-28 22:40:00,3.17,2.0,2.0,85.10,3.0,2,19:30:00


'2019-04-18 05:50:00'

In [6]:
from sklearn.linear_model import LinearRegression

y = train_df['price']
y_test = test_df['price']

y = y.fillna(y.mean())
y_test = y_test.fillna(y_test.mean())

X_test = test_df.drop(['price'], axis=1)
X = train_df.drop(['price'], axis=1)

reg = LinearRegression().fit(X, y)

In [7]:
pred = reg.predict(X_test)
pred_train = reg.predict(X)

from sklearn.metrics import mean_absolute_error


print("MAE (train): ", mean_absolute_error(y_test, pred))
print("MAE (train): ", mean_absolute_error(y, pred_train))

MAE (train):  12.736846154777966
MAE (train):  12.55922752691473


In [8]:
submission = submission.reindex(columns = submission.columns.tolist() + ['Price'])
submission['Price'] = pred                                                                 

submission

,id,Price
0,3002,83.554961
1,3003,83.948783
2,3004,83.050868
3,3005,27.963430
4,3006,83.554961
...,...,...
1994,4996,83.554961
1995,4997,83.554961
1996,4998,82.893340
1997,4999,82.893340


In [9]:
from google.colab import files

# Exportamos el CSV borrando el índice
submission.to_csv('submission_kaggle.csv', index=False) 
files.download('submission_kaggle.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>